# **Supervised Learning - Autism Dataset for Toddlers**

## Autism Spectrum Disorder (ASD) Diagnosis

# **Introduction**

# The Dataset being used is adapted from the Kaggle's Autism Dataset for Toddlers page, which contains 1054 reported cases , each with 19 attributes. The most important ones to look at is, namely, the Q-chat-score, that is , the aggregate of the results from the 10-question questionaire, that corresponds to influential features to be utilised for further analysis especially in determining autistic traits , each one corresponding also to an binary attribute, and other labeled attributes, such as Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test, and the "Class/ASD Traits ", which is derived from the Q-Chat Score. The remaining one that is numeric , is the case numeber, a unique identifier for each row of results

# The goal of this project is to improve the classification of the ASD traits, given a whole dataset of diagnosis based on certain features evaluated on the questionnaires, and evaluate their distribution by other parallel factors that are labeled, such as Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test, and the "Class/ASD Traits "

# The solution to this problem is a supervised learning model, which will be trained using the dataset mentioned above. The model will be trained using the training set, and then evaluated using the test set. The model will be evaluated using the accuracy metric, which is the percentage of diagnosis that are correctly done / probability of a certain diagnose is correctly done , taking into account all the labeled factors.

---

This project was made possible by:

| Name | Email |
|-|-|
| André Silva | up202108724@up.pt |
| Bernardo Pinto | up202108842@up.pt |
| Francisco Sousa | up202108838@up.pt |
|---|---|
| Group | T10 - G104 |

### Importing libraries

Throughout the study, many libraries were incrementally added, thus, it is important to install them all, which can be done by running the following command in the terminal (make sure you are in the project's root directory):

```bash
pip install -r requirements.txt
```

Then, we can import the libraries we will use in this project.

Note that we also had disabled the warnings, to make the notebook cleaner.

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn


ModuleNotFoundError: No module named 'sklearn'

In [16]:
#import pandas as pd

dataframe = pd.read_csv("./Autism_dataset.csv")
dataframe.head()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes


# Data pre-processing



In [18]:
dataframe.describe()


,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score
count,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000,1054.000000
mean,527.500000,0.563567,0.448767,0.401328,0.512334,0.524668,0.576850,0.649905,0.459203,0.489564,0.586338,27.867173,5.212524
std,304.407895,0.496178,0.497604,0.490400,0.500085,0.499628,0.494293,0.477226,0.498569,0.500128,0.492723,7.980354,2.907304
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000
25%,264.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,3.000000
50%,527.500000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,30.000000,5.000000
75%,790.750000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,36.000000,8.000000
max,1054.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,36.000000,10.000000


In [6]:
dataframe.isna().any()

Case_No                   False
A1                        False
A2                        False
A3                        False
A4                        False
A5                        False
A6                        False
A7                        False
A8                        False
A9                        False
A10                       False
Age_Mons                  False
Qchat-10-Score            False
Sex                       False
Ethnicity                 False
Jaundice                  False
Family_mem_with_ASD       False
Who completed the test    False
Class/ASD Traits          False
dtype: bool

In [21]:
#encoder = LabelEncoder()

#dataframe['"Class/ASD Traits "'] = encoder.fit_transform(dataframe['"Class/ASD Traits "'])
#dataframe.head()

NameError: name 'LabelEncoder' is not defined